In [1]:
#Retail & E-Commerce

# Instacart Market Basket Analysis
# Kaggle

#data
#https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis/data

#class data doc
#https://docs.google.com/document/d/1BsJIWU4TKUkADvnNEC14b8coQ5VqslJtS7ZBCYJVZAk/edit?tab=t.ta49d25hofw5#heading=h.bbadcysas44j

# Week 2 - Preprocessing, part 2

# 1. Lesson: None

# 2. Weekly graph question

The Storytelling With Data book mentions planning on a "Who, What, and How" for your data story.  Write down a possible Who, What, and How for your data, using the ideas in the book.

In [2]:
'''
who: business leaders wanting to sell products before expire or easons?
what: we want to sell more of x product
how: ill show what products or areas could be better
'''

'\nwho: business leaders wanting to sell products before expire or easons?\nwhat: we want to sell more of x product\nhow: ill show what products or areas could be better\n'

# 3. Homework - work with your own data

This week, you will do the same types of exercises as last week, but you should use your chosen datasets that someone in your class found last semester. (They likely will not be the particular datasets that you found yourself.)

### Here are some types of analysis you can do  Use Google, documentation, and ChatGPT to help you:

- Summarize the datasets using info() and describe()

- Are there any duplicate rows?

- Are there any duplicate values in a given column (when this would be inappropriate?)

- What are the mean, median, and mode of each column?

- Are there any missing or null values?

    - Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?

- Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)

- Encode any categorical variables (e.g. with one-hot encoding.)

### Conclusions:

- Are the data usable?  If not, find some new data!

- Do you need to modify or correct the data in some way?

- Is there any class imbalance?  (Categories that have many more items than other categories).

<hr>


#### Load Data & Libs

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import glob
import os

In [ ]:


base = "../data/Retail_and_ECommerce/Instacart_Market_Basket_A"

# ---- Simple files ----
aisles = pd.read_csv(os.path.join(base, "aisles.csv"))
departments = pd.read_csv(os.path.join(base, "departments.csv"))
products = pd.read_csv(os.path.join(base, "products.csv"))
order_products_train = pd.read_csv(os.path.join(base, "order_products__train.csv"))

# ---- Combine split files ----

# orders parts
orders_files = glob.glob(os.path.join(base, "orders_part_*.csv"))
orders = pd.concat([pd.read_csv(f) for f in orders_files], ignore_index=True)

# prior parts
prior_files = glob.glob(os.path.join(base, "order_products__prior_*.csv"))
order_products_prior = pd.concat([pd.read_csv(f) for f in prior_files], ignore_index=True)

print("Loaded everything.")
print("orders shape:", orders.shape)
print("order_products_prior shape:", order_products_prior.shape)


Loaded everything.
orders shape: (3421083, 7)
order_products_prior shape: (32434489, 4)


: 

##### Merge the data

In [ ]:
# Combine prior and train if you want full dataset
order_products = pd.concat(
    [order_products_prior, order_products_train],
    axis=0
)

# Merge with orders
df = order_products.merge(orders, on="order_id", how="left")

# Merge with products
df = df.merge(products, on="product_id", how="left")

# Merge with aisles
df = df.merge(aisles, on="aisle_id", how="left")

# Merge with departments
df = df.merge(departments, on="department_id", how="left")

df.head()

#### Save merged dataset

In [ ]:
df.to_csv(f"{base}/Instacart_Market_Basket_Data_Merged.csv", index=False)

In [ ]:
#df = pd.read_csv(os.path.join(base, "Instacart_Market_Basket_Data_Merged.csv"))

#### Begin EDA

In [ ]:
#Summarize the datasets using info() and describe()

df.info()

df.describe()

In [ ]:
#Are there any duplicate rows?



In [ ]:
#Are there any duplicate values in a given column (when this would be inappropriate?)



In [ ]:
#What are the mean, median, and mode of each column?



In [ ]:
#Are there any missing or null values?
####Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?



In [ ]:
#Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)


In [ ]:
#Encode any categorical variables (e.g. with one-hot encoding.)

# 4. Storytelling With Data graph

Just like last week: choose any graph in the Introduction of Storytelling With Data. Use matplotlib to reproduce it in a rough way. I don't expect you to spend an enormous amount of time on this; I understand that you likely will not have time to re-create every feature of the graph. However, if you're excited about learning to use matplotlib, this is a good way to do that. You don't have to duplicate the exact values on the graph; just the same rough shape will be enough.  If you don't feel comfortable using matplotlib yet, do the best you can and write down what you tried or what Google searches you did to find the answers.